In [1]:
import sounddevice as sd
import librosa
import numpy as np
import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

In [2]:
def extract_features(signal, sr):
    features = {}
    
    # Fundamental frequency features
    pitches = librosa.yin(signal, fmin=50, fmax=300, sr=sr)
    features['MDVP:Fo(Hz)'] = pitches.mean()
    features['MDVP:Fhi(Hz)'] = pitches.max()
    features['MDVP:Flo(Hz)'] = pitches.min()
    
    # Jitter features
    features['MDVP:Jitter(%)'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:Jitter(Abs)'] = np.mean(np.abs(signal))
    features['MDVP:RAP'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:PPQ'] = np.sqrt(np.mean(signal ** 2))
    features['Jitter:DDP'] = features['MDVP:RAP'] * 3
    
    # Shimmer features
    features['MDVP:Shimmer'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:Shimmer(dB)'] = np.sqrt(np.mean(signal ** 2))
    features['MDVP:APQ'] = np.sqrt(np.mean(signal ** 2))
    features['Shimmer:DDA'] = features['MDVP:APQ'] * 3
    features['shimmer:APQ3'] = np.sqrt(np.mean(signal ** 2))
    features['shimmer:APQ5'] = np.sqrt(np.mean(signal ** 2))

    # Other features
    features['NHR'] = np.sqrt(np.mean(signal ** 2))
    features['HNR'] = np.sqrt(np.mean(signal ** 2))
    features['RPDE'] = np.sqrt(np.mean(signal ** 2))
    features['DFA'] = np.sqrt(np.mean(signal ** 2))
    features['spread1'] = -np.sqrt(np.mean(signal ** 2))
    features['spread2'] = np.sqrt(np.mean(signal ** 2))
    features['D2'] = np.sqrt(np.mean(signal ** 2))
    features['PPE'] = np.sqrt(np.mean(signal ** 2))
    
    return features


In [3]:
# Set the sample rate for recording and processing the audio
sample_rate = 22050  # Sample rate in Hz
duration = 10  # Duration in seconds

In [4]:
# Record audio from the microphone
print('Recording audio...')
recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1)
sd.wait()  # Wait for the recording to complete


Recording audio...


In [5]:
# Flatten the recording array to one-dimensional
signal = recording.flatten()

In [6]:
# Extract features from the recorded audio
extracted_features = extract_features(signal, sample_rate)

In [7]:
# Print the extracted features
for feature, value in extracted_features.items():
    print(f'{feature}: {value}')

MDVP:Fo(Hz): 152.4951521323191
MDVP:Fhi(Hz): 302.05479452054794
MDVP:Flo(Hz): 50.0
MDVP:Jitter(%): 0.05729587748646736
MDVP:Jitter(Abs): 0.036973778158426285
MDVP:RAP: 0.05729587748646736
MDVP:PPQ: 0.05729587748646736
Jitter:DDP: 0.17188763245940208
MDVP:Shimmer: 0.05729587748646736
MDVP:Shimmer(dB): 0.05729587748646736
MDVP:APQ: 0.05729587748646736
Shimmer:DDA: 0.17188763245940208
shimmer:APQ3: 0.05729587748646736
shimmer:APQ5: 0.05729587748646736
NHR: 0.05729587748646736
HNR: 0.05729587748646736
RPDE: 0.05729587748646736
DFA: 0.05729587748646736
spread1: -0.05729587748646736
spread2: 0.05729587748646736
D2: 0.05729587748646736
PPE: 0.05729587748646736


In [10]:
feature_vector = []
feature_order = ['MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'MDVP:RAP',
                 'MDVP:PPQ', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'MDVP:APQ', 'Shimmer:DDA','shimmer:APQ3','shimmer:APQ5',
                 'NHR', 'HNR', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']

for feature in feature_order:
    feature_vector.append(extracted_features[feature])

print(feature_vector)

[152.4951521323191, 302.05479452054794, 50.0, 0.057295877, 0.03697378, 0.057295877, 0.057295877, 0.17188763245940208, 0.057295877, 0.057295877, 0.057295877, 0.17188763245940208, 0.057295877, 0.057295877, 0.057295877, 0.057295877, 0.057295877, 0.057295877, -0.057295877, 0.057295877, 0.057295877, 0.057295877]


In [15]:
import csv
# Read the CSV file and store its contents in an array of arrays with elements as floats
X_test = []
with open('svminput.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        row_float = [float(element) for element in row]
        X_test.append(row_float)

# Print the array of arrays
for row in X_test:
    print(row)

[-1.700085828209211, -0.9679684096448317, -0.7701302154312807, -0.2750006827910493, 0.4161566828723056, -0.29261511273205904, -0.09708697831590672, -0.29162165542639146, -0.4947066562822746, -0.490058396079976, -0.5324881711161069, -0.4268488537203282, -0.36025142239292546, -0.5324846875615613, -0.3571897128986134, -0.10884033692354644, 1.0696370502274182, 1.05628303643319, 0.37218019866695545, 1.948862077042917, 0.036693507080304166, 0.44431448197637585]
[-1.3904409510920863, -0.929681132475558, -0.7370456769440031, 0.742068829383586, 1.5045128023707084, 0.8543498191074901, 0.7336398622306344, 0.853234750777349, -0.003125385615247059, 0.3016600937728975, 0.11651101109799405, -0.07675951489648845, -0.22396741269635478, 0.11682927573696351, -0.11964497356176824, -0.5227908336786481, 0.912650090252046, 1.3172199457952372, 0.670118138152274, 0.4743186080828963, 0.014245486786233172, 0.7468597989637795]
[-1.3530206527639494, -0.6291752917367951, -0.7290272246426058, 0.4920948967477938, 1.2

In [16]:
scaler = StandardScaler()
scaler.fit(X_test)


StandardScaler()

In [18]:
input_data_as_numpy_array = np.asarray(feature_vector)

# reshape the numpy array
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the data
std_data = scaler.transform(input_data_reshaped)
print(std_data)

[[ 1.38811538e+02  2.89217305e+02  5.95507934e+01  6.98792515e-01
   2.04808282e-01  7.32203286e-01  5.66247147e-01  9.89708106e-01
   7.65934759e-01  8.50998344e-01  7.27579588e-01  8.22348199e-01
   8.96024729e-01  7.27526519e-01  1.51346344e+00 -3.30150619e-01
  -1.67551082e-02  1.76367985e-01  1.17045855e-02 -9.53990890e-03
   2.46150843e-01  1.89726642e-01]]


In [23]:
svm_model = joblib.load('SVM_model.pkl')
prediction = svm_model.predict(std_data)
print(prediction)

[0]
